In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import logging
import requests
import datetime as dt
import pylab

import scipy.stats as stats
from matplotlib import pyplot

import numpy as np

import warnings
warnings.filterwarnings('ignore')

logging.getLogger().setLevel(logging.INFO)


from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os, sys

In [2]:
home = os.environ["HOME"]
sys.path.append(f"{home}/Desktop/project-ursa/lib")

import imp

In [4]:
import monte_carlo
imp.reload(monte_carlo)

import data_utils
imp.reload(data_utils)

# import data_analysis
# imp.reload(data_analysis)

import mining_constants
imp.reload(mining_constants)

import machine_npv
imp.reload(machine_npv)

<module 'machine_npv' from '/Users/merkle-employee/Desktop/project-ursa/lib/machine_npv.py'>

In [7]:
du = data_utils.DataUtility("cc_key")

##
du._raw_network_price_df = pd.read_csv("../../data/raw_network_price.csv").drop("Unnamed: 0", axis="columns")
du._raw_network_price_df.time = pd.to_datetime(du._raw_network_price_df.time)
du._df_feature() 
du._aggregate_weekly_df() 

daily_df  = du._daily_network_price_df
weekly_df = du._weekly_network_price_df

## 
sp = monte_carlo.GBMPriceLinearHashRate()
sp.fit(weekly_df, sample_rate=mining_constants.SAMPLE_RATE_WEEKLY)

## 
k = len(weekly_df) - 1

start_time = weekly_df.reset_index().loc[k, "time"]
hash_rate  = weekly_df.reset_index().loc[k, "hashrate"]
hash_index = weekly_df.reset_index().loc[k, "hash_index"]
btc_close  = weekly_df.reset_index().loc[k, "close"]

print("week: ", start_time, "hashrate: ", hash_rate, "hash-index: ", hash_index, "btc-close: ", btc_close)


week:  2023-04-14 hashrate:  346035351.07423794 hash-index:  0.07690399113505421 btc-close:  29568.332857142854


In [9]:
machine_params   = mining_constants.machine_params()
machine_duration = 1.2
ky = "S19J"
params = {
    "asic_energy_consumption" : machine_params[ky]["energy_consumption"],
    "asic_hash_rate"          : machine_params[ky]["hash_rate"],
    "electricity_cost"        : 0.05,
    "machine_duration"        : machine_duration,
    "sample_rate"             : mining_constants.SAMPLE_RATE_WEEKLY,
    "mc_size"                 : 1000,
    "interest_rate"           : 0.05,
    "asic_number"             : 1,
    "asic_efficiency"         : 1.0,
    "pool_fee"                : 0.005,
    "analysis_denomination"   : mining_constants.ASSET_USD
}

import machine_npv
imp.reload(machine_npv)

m_npv = machine_npv.MachineNPV(sp, params)
m_npv._setup(start_time, btc_close, hash_rate)

m_usd_bsm = machine_npv.MachineNPV_USD_BSM(sp, params)
usd_bsm = m_usd_bsm.price(start_time, btc_close, hash_rate)

m_usd_bin = machine_npv.MachineNPV_USD_Bin(sp, params)
usd_bin = m_usd_bin.price(start_time, btc_close, hash_rate, 100)

m_btc_bin = machine_npv.MachineNPV_BTC_Bin(sp, params)
btc_bin = m_btc_bin.price(start_time, btc_close, hash_rate, 100)

In [12]:
usd_bsm, usd_bin, usd_bin / btc_close, btc_bin

(1048.5823947443246,
 1048.7237385009564,
 0.035467800757242054,
 0.02742934663795226)

In [13]:
btc_bin

0.02742934663795226